In [ ]:
# Importar pacotes
import kagglehub
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import timedelta

# Plotar gráficos
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import missingno as msno

# Combinaçao entre variaveis para plot em EDA
from itertools import combinations

# trabalhar com arquivo yaml
import yaml

# Save and load Modelos
import joblib

# Normalização de Escala
from sklearn.preprocessing import MinMaxScaler

# Redução de dimensionalidade
from sklearn.decomposition import PCA

# VarianceThreshold
from sklearn.feature_selection import VarianceThreshold

# Preparar para treinamentos
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

# Modelos de Predição
from sklearn import linear_model
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor

# Métricas/Score de regresssão
from sklearn import metrics
import scipy.stats as stats

In [ ]:
# definir parametros de Confif YAML
with open("./config/conf.yaml", 'r') as stream:
    conf_features = yaml.safe_load(stream)

# Definição de variváveis
target_prediction = conf_features['target_prediction']
iron_concentrate = conf_features['iron_concentrate']

try:
    colunas = conf_features['colunas']
except:
    pass


## Dados

### Raw Data
##### - Não é preciso processar todas as vezes
##### - Se desejar, pode seguir para Preprocessed Data

In [ ]:
# Download latest version from KaggleHub
path = kagglehub.dataset_download("edumagalhaes/quality-prediction-in-a-mining-process")

data = pd.read_csv(Path(path).joinpath(conf_features['rawdata_filename']), sep=',', header=0)

# salvar na pasta do projeto
data.to_csv('./data/01_raw/' + conf_features['rawdata_filename'], index=False)

In [ ]:
# Leitura de Raw Data
data = pd.read_csv('./data/01_raw/' + conf_features['rawdata_filename'], sep=',', header=0)

for column in data.columns:
    data[column] = data[column].apply(lambda x : x.replace(',' , '.'))

data['date'] = pd.to_datetime(data['date'])

data.set_index('date', inplace = True)

data = data.astype('double')

data

In [ ]:
# definir as colunas do dataset
colunas = list(data.columns)

conf_features['colunas'] = colunas

with open("./config/conf.yaml", 'w') as outfile:
    yaml.dump(conf_features, outfile, default_flow_style=False)

colunas

In [ ]:
# vizualizar resumo de informações sobre o dataset
print(data.info())

In [ ]:
# ajustar date timestamp 
data.reset_index(inplace=True)

# Criação do DataFrame
lista_datas = data['date'].unique()
for date_time in lista_datas:
    size_df = data[data['date'] == date_time]['date'].size
    df = pd.DataFrame(pd.date_range(date_time, date_time + timedelta(hours=1), freq="20s", name = 'date'))
    data.loc[data['date'] == date_time, 'date'] = (df[-size_df:] - timedelta(seconds=20)).values

data.set_index('date', inplace = True)
data

In [ ]:
# Plotar variável de interesse (target) na linha do tempo
plt.figure(figsize=(18,6))
plt.scatter(data.index, data[target_prediction])
plt.title(target_prediction)

In [ ]:
# Criar Dataset corrigindo 'date' com timestamp
df = pd.DataFrame(pd.date_range(data.index.min(), data.index.max(), freq="20s", name = 'date'))
df = pd.concat([df.set_index('date'), data], axis=1)
df

### Preprocessed Data

In [ ]:
# Salver Dataset Preprocessado
df.to_csv('./data/02_preprocessed/dataset.csv', sep = ';')

# Load Dataset
df = pd.read_csv('./data/02_preprocessed/dataset.csv', sep = ';')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace = True)
df

In [ ]:
# Visualização de Missing Data
msno.matrix(df, freq = 'M')

In [ ]:
# Filtrar data em periodo que houve operação contínua
df = df[df.index> df[df[target_prediction].isna()].index[-1]]
df

In [ ]:
# Salver Dataset Time Filtered
df.to_csv('./data/02_preprocessed/dataset_timefiltered.csv', sep = ';')

# Load Dataset
df = pd.read_csv('./data/02_preprocessed/dataset_timefiltered.csv', sep = ';')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace = True)
df

## Preparação de Features

#### - Vizualização do dataset antes de tratamento para Features

In [ ]:
# Plotar histogramas das variáveis, 
# Finalidade em oberservar o comportamento de suas distribuição
for col in colunas:
    fig = px.histogram(df, x = col)
    fig.show()

In [ ]:
for col in colunas:
  plt.figure(figsize=(18,6))
  fig = plt.scatter(df.index, df[col])
  plt.title(col)
  plt.show()

In [ ]:
# Utilizar uma janela de 1 hora para amortecer ruído do dado e do processo 
window_size = 180

for coluna in colunas:

    # feature media movel
    df.loc[:, coluna] = df[coluna].rolling(window_size).mean()

    # feature para observar/filtrar dados com altavariabilidade e transição de operação / operação estável
    df[f'{coluna}-std'] = ''
    df.loc[:, f'{coluna}-std'] = df[coluna].rolling(window_size).std()

df.dropna(inplace = True)
df

In [ ]:
# Salver Dataset de Features
df.to_csv('./data/03_features/preliminary_feature_dataset.csv', sep = ';')

# Load Dataset
df = pd.read_csv('./data/03_features/preliminary_feature_dataset.csv', sep = ';')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace = True)
df

### Análise das Features
#### - Análise Exploratória de Dados (EDA)

In [ ]:
# Descrição estatísitca
df.describe()

In [ ]:
# Plotar histogramas das variáveis, 
# Finalidade em oberservar o comportamento de suas distribuição
for col in colunas:
    fig = px.histogram(df, x = col)
    fig.show()

In [ ]:
for col in colunas:
  plt.figure(figsize=(18,6))
  fig = plt.scatter(df.index, df[col])
  plt.title(col)
  plt.show()

In [ ]:
# filtra instabilidade no processo pela variabilidade de air flow
df = df[df['Flotation Column 01 Air Flow-std']<0.3]
df = df[df['Flotation Column 02 Air Flow-std']<0.3]
df = df[df['Flotation Column 03 Air Flow-std']<0.3]
df = df[df['Flotation Column 04 Air Flow-std']<0.3]
df = df[df['Flotation Column 05 Air Flow-std']<0.3]
df = df[df['Flotation Column 06 Air Flow-std']<0.3]
df = df[df['Flotation Column 07 Air Flow-std']<0.3]

# filtra instabilidade no processo pela variabilidade de Level
df = df[df['Flotation Column 01 Level-std']<3]
df = df[df['Flotation Column 02 Level-std']<3]
df = df[df['Flotation Column 03 Level-std']<3]
df = df[df['Flotation Column 04 Level-std']<3]
df = df[df['Flotation Column 05 Level-std']<3]
df = df[df['Flotation Column 06 Level-std']<3]
df = df[df['Flotation Column 07 Level-std']<3]

In [ ]:
# Salver Dataset de Features
df[colunas].to_csv('./data/03_features/feature_dataset.csv', sep = ';')

# Load Dataset
df = pd.read_csv('./data/03_features/feature_dataset.csv', sep = ';')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace = True)
df

In [ ]:
# Plotar histogramas das variáveis, 
# Finalidade em oberservar o comportamento de suas distribuição
for col in colunas:
    fig = px.histogram(df, x = col)
    fig.show()

In [ ]:
# Salvar alguns gráficos para análise de correlação de variávies
for gx, gy in [['% Silica Feed', target_prediction],
               ['% Iron Feed', '% Iron Concentrate'],
               ['% Iron Concentrate', target_prediction],
               ['% Iron Feed', '% Silica Feed'],
               ['% Silica Feed', '% Iron Concentrate'],
              ]:
    # plotar grafico realcionando 2 variáveis
    fig = plt.figure(figsize = (8,6))
    sns.scatterplot(df,
                    x = gx, 
                    y = gy,
                    hue = target_prediction,
                    palette = 'viridis',
                    linewidth=0.1
                )
    fig.figure.savefig(f"./EDA Results/Scatter Plot/{gx}x{gy}.png")
    plt.show()

In [ ]:
# for gx, gy in combinations(df[colunas].drop(columns=target_prediction), 2):
#     # plotar grafico realcionando 2 variáveis
#     fig = plt.figure(figsize = (8,6))
#     sns.scatterplot(df,
#                     x = gx, 
#                     y = gy,
#                     hue = target_prediction,
#                     palette = 'viridis',
#                     linewidth=0.1
#                 )
#     # fig.figure.savefig(f"./EDA Results/Scatter Plot/{gx}x{gy}.png")
#     plt.show()

In [ ]:
# Correlação entre features
plt.figure(figsize=(14,12))
plot = sns.heatmap(df[colunas].corr(method = 'pearson'),annot=True,fmt=".2f", linewidth=.5)
plt.show()
plot.figure.savefig(f"./EDA Results/feature_correlation.png")

# Treinamento de Modelos

## Regressão
#### Previsão de % Silica Concentrate'

In [ ]:
# Preparando o dataset para executar treinamento de predição/regressão
# Colunas das variáveis utilizadas na prediçao/regressão 
# (deve avaliar qualidade do modelo sem a variavel % Iron Concentrate, pois este também um resultado do processo)
Y = df[target_prediction]
X = df[colunas].drop(columns=[target_prediction, iron_concentrate])

# Incluir features no arquivo de configuração
conf_features['features'] = list(X.columns)
with open("./config/conf.yaml", 'w') as outfile:
    yaml.dump(conf_features, outfile, default_flow_style=False)

# selecionar apenas as features desejadas para o treinmaento
X = X[conf_features['features']]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [ ]:
# organizaçao de nomes dos modelos testados
names = [
    "Linear",
    "Ridge",
    "LARS Lasso",
    # "SRV", # SRV não apresentou treinamento satisfatorio, ainda com longo tempo de treinamento
    "RandomForestRegressor",
    "GradientBoostingRegressor",
    "AdaBoostRegressor"
]

# lista dos modelos testados
regression_models = [
    linear_model.LinearRegression(),
    linear_model.Ridge(alpha=.5),
    linear_model.LassoLars(alpha=.1),
    # svm.SVR(),
    RandomForestRegressor(max_depth=10, random_state=42),
    GradientBoostingRegressor(random_state=42),
    AdaBoostRegressor(random_state=42, n_estimators=100)
]

In [ ]:
# metrica para valiar razao entre F1-score
f1score_ration = pd.DataFrame([], columns=names)

# Iteração entre cada modelo
for name, reg_model in zip(names, regression_models):

    # # MinMaxScaler
    # # Possibilidade de trabalhar com ajustando dimensão e escala
    scaler = MinMaxScaler()
    
    # # Variance Threshold
    # # Possibilidade de trabalhar com VarianceThreshold
    variance_selector = VarianceThreshold()

    # # PCA
    # # Possibilidade de trabalhar com reduçao de dimensionaldiade
    pca = PCA(n_components=10)
    
    # criando pipeline com modelo e feature selection
    pipeline = Pipeline([
        # ('scaler', scaler),
        # ('variance', variance_selector),
        # ('pca', pca),
        ('regression_model', reg_model)
    ])

    # realizando o treinmaneto da classificação
    reg = pipeline.fit(X_train, y_train)
    joblib.dump(reg, f'./models/prediction_{name}.joblib')

    # capitura de score nos dados de teste
    y_pred_train = reg.predict(X_train)
    y_pred_test = reg.predict(X_test)
    score_train = reg.score(X_train, y_train)
    score_test = reg.score(X_test, y_test)
    
    # printar os resultados do modelo de classificação
    print(f'======================={name}==================================')
    print('Train')
    print('Mean Absolute Error (MAE) = ', metrics.mean_absolute_error(y_train, y_pred_train))
    print('Mean Squared Error (MSE) = ',  metrics.mean_squared_error(y_train, y_pred_train))
    print('Root Mean Squared Error (RMSE) = ', metrics.root_mean_squared_error(y_train, y_pred_train))
    print('R-squared (R2) Score = ', score_train)
    print('')
    print('Test')
    print('Mean Absolute Error (MAE) = ', metrics.mean_absolute_error(y_test, y_pred_test))
    print('Mean Squared Error (MSE) = ',  metrics.mean_squared_error(y_test, y_pred_test))
    print('Root Mean Squared Error (RMSE) = ', metrics.root_mean_squared_error(y_test, y_pred_test))
    print('R-squared (R2) Score = ', score_test)
    # residuos apresentão distribuição normal - teste de Shapiro-Wilk
    display(stats.shapiro(y_test - y_pred_test))
    display(stats.anderson(y_test - y_pred_test))

    plt.figure(figsize = (8,6))
    sns.histplot(y_test - y_pred_test).set_title(name)
    plt.show()

### Load Model
#### - Modelo Escolhido é Random Forest

In [ ]:
# Load de model
reg = joblib.load('./models/prediction_RandomForestRegressor.joblib')
y_pred_test = reg.predict(X_test)
y_pred_test

In [ ]:
df_plot = pd.DataFrame({'Valor Verdadeiro': y_test,
                        'Valor Predito': reg.predict(X_test)})

plt.figure(figsize = (8,6))
sns.regplot(df_plot,
                x = 'Valor Verdadeiro',
                y = 'Valor Predito',
                scatter_kws=dict(alpha=0.15),
                line_kws=dict(color="r", alpha=0.4)
                ).set_title('Random Forest Teste')

In [ ]:
feature_importance = pd.DataFrame({'feature': X.columns,
                                   'importance': reg.named_steps['regression_model'].feature_importances_})
most_import_feature = feature_importance.sort_values(by='importance', ascending=False)
most_import_feature[:9]

In [ ]:
# Se deseja reduzir features de maior importancia no arquivo de configuração
# conf_features['features'] = list(most_import_feature[:9]['feature'])
# with open("./config/conf.yaml", 'w') as outfile:
#     yaml.dump(conf_features, outfile, default_flow_style=False)

In [ ]:
plt.figure(figsize = (12,8))
sns.barplot(feature_importance.sort_values(by='importance', ascending=False),
            x = 'importance', 
            y = 'feature',
            hue = 'importance',
            palette='viridis')

# Analise de Cluster

In [ ]:
# from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import SpectralClustering
from sklearn.cluster import DBSCAN
from sklearn.cluster import HDBSCAN
from sklearn.cluster import OPTICS
from sklearn.cluster import AgglomerativeClustering

from sklearn.metrics import silhouette_samples, silhouette_score

from sklearn.manifold import TSNE

In [ ]:
# Cenario 1 
## Análise de Clusters com qualiadde de entrara e sáida do processo
X_cluster = ['% Iron Feed',
             '% Silica Feed',
             '% Iron Concentrate',
             '% Silica Concentrate']

data_cluster = df[X_cluster]

In [ ]:
# Cenario 2
## Análise de Clusters com dados de variaveis mais importantes da regressão
X_cluster = list(most_import_feature[:9]['feature'])
X_cluster.append(iron_concentrate)
X_cluster.append(target_prediction)

data_cluster = df[X_cluster]

X_cluster

In [ ]:
results = dict()
k_cand = [2,3,4,5,6,7]

for k in k_cand:
    kmeans = KMeans(n_clusters=k, random_state=0).fit(data_cluster)
    score0 = kmeans.inertia_
    score1 = silhouette_score(data_cluster, kmeans.labels_, metric='euclidean')
    score2 = silhouette_score(data_cluster, kmeans.labels_, metric='correlation')
    results[k] = {'k':kmeans, 
                  's0':score0, 
                  's1':score1, 
                  's2':score2}
    display(results[k])

fig,axs = plt.subplots(1,2,sharex=True,figsize=(10,3))
axs[0].plot([i for i in results.keys()], [i['s0'] for i in results.values()], 'o-', label='Inertia')
axs[1].plot([i for i in results.keys()], [i['s1'] for i in results.values()], 'o-', label='Euclidean')
axs[1].plot([i for i in results.keys()], [i['s2'] for i in results.values()], 'o-', label='Correlation')
for ax in axs:
    ax.set_xticks(k_cand)
    ax.set_xlabel('K')
    ax.legend()

In [ ]:
clustering = kmeans
df2 = pd.DataFrame(clustering.labels_)
df2 = df2.rename(columns = {0: 'valor'})
df2['cont'] = 1
df2.groupby(by = ['valor']).describe()

In [ ]:
# SpectralClustering não performou, dataset muito grande
# calculating the distance between every vector in your dataset and every other vector
# clustering = SpectralClustering(n_clusters=4,
#         assign_labels='discretize',
#         random_state=0).fit(data_cluster)

# clustering = DBSCAN(eps=100, min_samples=20).fit(data_cluster)

# clustering = HDBSCAN(copy=True, min_cluster_size=20).fit(data_cluster)

clustering = OPTICS(min_samples=20).fit(data_cluster)

# clustering = AgglomerativeClustering(n_clusters=20).fit(data_cluster)

# análise dos clusters
df2 = pd.DataFrame(clustering.labels_)
df2 = df2.rename(columns = {0: 'valor'})
df2['cont'] = 1
df2.groupby(by = ['valor']).describe()

In [ ]:
# data_cluster.drop(columns =['level_0', 'index'], inplace=True)

In [ ]:
# data_cluster[target_prediction] = df[target_prediction]
lista = data_cluster.columns
lista

In [ ]:
data_cluster.loc[:,'group'] = clustering.labels_.astype(str)

In [ ]:
data_plot = data_cluster.reset_index().melt(id_vars=['date','group'], var_name='Variavel', value_name='Value')

In [ ]:
data2 = data_cluster

data_cluster.reset_index(inplace=True)

data_cluster['date'] = pd.to_datetime(data_cluster['date'])

# data2.reset_index(inplace = True)

data2.loc[:,'month'] = data2['date'].dt.year.astype(str) + '/' + data2['date'].dt.month.map("{:02}".format).astype(str)
data2.loc[:,'day'] = data2['date'].dt.day

data2.loc[:,'date'] = pd.to_datetime(data2['date']).dt.to_period('M').astype(str)
data2.loc[:,'count'] = 1
data2

In [ ]:
px.bar(data2[['date', 'month', 'group', 'count']].groupby(['date', 'group']).sum().reset_index().sort_values(by = 'group'),
      x = 'date',
      y = 'count',
      color = 'group',
      barmode = 'group')

In [ ]:
# plotar box plot, entre mêses
for col in lista:
    fig = px.box(data_cluster.sort_values(by = 'group'),
           x = 'group',
           y = col,
           color = 'group'
                )
    fig.show()

## Analise de Monte Carlo
### - Simular ceários de operação, de tal forma a entender as tomasdas de decisões

In [ ]:
X = list(most_import_feature['feature'])
X.append(target_prediction)

data_simulation = df[X]
data_simulation.loc[:, 'group'] = clustering.labels_.astype(str)
data_simulation

In [ ]:
cluster_from = '6'
grupo = data_simulation[data_simulation['group'] == cluster_from ]

cluster_to = '2'
parameters_toadjust = ['Amina Flow',
                      'Flotation Column 04 Air Flow',
                      'Flotation Column 02 Level',
                      'Flotation Column 06 Level',
                      'Flotation Column 07 Level'
                      ]
grupo_adjust = data_simulation[data_simulation['group'] == cluster_to ]


In [ ]:
X = list(most_import_feature.sort_index()['feature'])
Y = target_prediction

In [ ]:
dic = { col: np.random.uniform(grupo.describe()[col]['25%'], grupo.describe()[col]['75%'], 1000) for col in X}
dic

In [ ]:
df_carlo = pd.DataFrame(dic)
df_carlo

In [ ]:
df_carlo_adjusted = df_carlo.copy()

for col in parameters_toadjust:
    df_carlo_adjusted.loc[:,col] = np.random.uniform(grupo_adjust.describe()[col]['25%'], grupo_adjust.describe()[col]['75%'], 1000)

df_carlo_adjusted

In [ ]:
plot_y_data = pd.DataFrame([], columns = ['Distribuição', 'Valores'])
plot_y_data['Valores'] = df[Y]
plot_y_data['Distribuição'] = 'Real Global'
plot_y_data

aux = pd.DataFrame([], columns = ['Distribuição', 'Valores'])
aux['Valores'] = grupo[Y]
aux['Distribuição'] = f'Real Grupo {cluster_from}'

plot_y_data = pd.concat([plot_y_data, aux])
plot_y_data


aux = pd.DataFrame([], columns = ['Distribuição', 'Valores'])
aux['Valores'] = reg.predict(df_carlo[X])
aux['Distribuição'] = 'Monte Carlo Random Forest'

plot_y_data = pd.concat([plot_y_data, aux])
plot_y_data


aux = pd.DataFrame([], columns = ['Distribuição', 'Valores'])
aux['Valores'] = reg.predict(df_carlo_adjusted[X])
aux['Distribuição'] = 'Random Forest - Parâmetros Ajustados'

plot_y_data = pd.concat([plot_y_data, aux])
plot_y_data


In [ ]:
px.box(
    plot_y_data, 
    x = 'Distribuição',
    y = 'Valores',
    color = 'Distribuição'
       )

In [ ]:
grupo.describe()

In [ ]:
df_carlo.describe()

In [ ]:
df_carlo_adjusted.describe()

# Fim !